[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_23/VL23_Self_Supervised_Learning_Demo.ipynb)

# Self-Supervised Learning Demo
## Vorlesung 23: Contrastive Learning mit CIFAR-10

**Ziel:** Verstehen, wie Self-supervised Learning ohne Labels funktioniert

**Methode:** Contrastive Learning - das Modell lernt, dass verschiedene Augmentierungen desselben Bildes ähnlich sein sollten

**Dataset:** CIFAR-10 (aber wir ignorieren die Labels für das Pretraining!)

In [ ]:
# Bibliotheken importieren
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import random

# Für reproduzierbare Ergebnisse
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print("Bibliotheken erfolgreich importiert!")
print(f"PyTorch Version: {torch.__version__}")

## 1. Daten vorbereiten

**Wichtig:** Wir verwenden CIFAR-10, aber **ignorieren die Labels komplett** für das Self-supervised Learning!

In [ ]:
# Nur 3 CIFAR-10 Klassen für klarere Visualisierung
classes = ['Flugzeug', 'Auto', 'Vogel', 'Katze', 'Hirsch', 'Hund', 'Frosch', 'Pferd', 'Schiff', 'LKW']
selected_classes = [0, 1, 2]  # Flugzeug, Auto, Vogel
class_names = ['Flugzeug', 'Auto', 'Vogel']

# Augmentierungen für Contrastive Learning definieren
class ContrastiveTransform:
    """Erstellt zwei verschiedene Augmentierungen desselben Bildes"""
    def __init__(self):
        # Starke Augmentierungen für Self-supervised Learning
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
    
    def __call__(self, x):
        # Zwei verschiedene Augmentierungen des gleichen Bildes
        return self.transform(x), self.transform(x)

# Dataset laden (Labels werden ignoriert!)
full_train_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True, 
    download=True, 
    transform=ContrastiveTransform()
)

# Nur ausgewählte Klassen filtern
def filter_classes(dataset, selected_classes):
    """Filtert Dataset um nur bestimmte Klassen zu behalten"""
    indices = []
    # Temporäres Dataset ohne Augmentierung für Label-Zugriff
    temp_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, transform=transforms.ToTensor()
    )
    
    for i in range(len(temp_dataset)):
        _, label = temp_dataset[i]
        if label in selected_classes:
            indices.append(i)
    
    return torch.utils.data.Subset(dataset, indices[:1000])  # Maximal 1000 Bilder

# Gefilterte Datasets erstellen
train_dataset = filter_classes(full_train_dataset, selected_classes)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

print(f"Dataset gefiltert: {len(train_dataset)} Bilder aus 3 Klassen für Self-supervised Learning")
print("WICHTIG: Wir verwenden KEINE Labels - nur die Bilder!")
print(f"Klassen: {', '.join(class_names)}")

## 2. Augmentierungen visualisieren

**Kernidee:** Das Modell lernt, dass diese verschiedenen Versionen desselben Bildes zusammengehören

In [ ]:
def show_augmentations():
    """Zeigt Original und Augmentierungen"""
    # Original Dataset für Vergleich
    original_dataset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=True, 
        transform=transforms.ToTensor()
    )
    
    fig, axes = plt.subplots(3, 3, figsize=(10, 8))
    
    # Zeige nur 3 Beispiele (einen pro Klasse)
    class_examples = []
    for target_class in selected_classes:
        for i in range(len(original_dataset)):
            _, label = original_dataset[i]
            if label == target_class:
                class_examples.append(i)
                break
    
    # Erstelle temporäres Dataset für konsistente Augmentierungen
    temp_augment_dataset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=True, 
        transform=ContrastiveTransform()
    )
    
    for col, idx in enumerate(class_examples):
        # Original Bild
        original_img, label = original_dataset[idx]
        axes[0, col].imshow(np.transpose(original_img, (1, 2, 0)))
        axes[0, col].set_title(f'Original: {classes[label]}')
        axes[0, col].axis('off')
        
        # Augmentierungen vom gleichen Bild
        aug1, aug2 = temp_augment_dataset[idx][0]  # Gleicher Index wie Original
        
        # Denormalisieren für Anzeige
        def denorm(tensor):
            mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
            std = torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
            return torch.clamp(tensor * std + mean, 0, 1)
        
        axes[1, col].imshow(np.transpose(denorm(aug1), (1, 2, 0)))
        axes[1, col].set_title('Augmentierung 1')
        axes[1, col].axis('off')
        
        axes[2, col].imshow(np.transpose(denorm(aug2), (1, 2, 0)))
        axes[2, col].set_title('Augmentierung 2')
        axes[2, col].axis('off')
    
    plt.suptitle('Self-Supervised Learning: 3 Klassen - Verschiedene Ansichten', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("💡 KERNIDEE: Das Modell lernt, dass die beiden Augmentierungen zusammengehören")
    print("   - Ähnliche Bilder → ähnliche Repräsentationen")
    print("   - Verschiedene Bilder → verschiedene Repräsentationen")

show_augmentations()

## 3. Einfaches Self-supervised Modell

**SimCLR-Ansatz:** Encoder + Projection Head für Contrastive Learning

### 💡 **Was passiert hier?**
- **Encoder**: CNN extrahiert Features aus Bildern (wie bei supervised Learning)
- **Projection Head**: Zusätzliches Netzwerk, das Features für Contrastive Learning optimiert
- **Kernidee**: Ähnliche Bilder sollen ähnliche Features haben, verschiedene Bilder verschiedene Features
- Das Modell lernt **ohne Labels** nur durch Vergleich von Augmentierungen!

In [ ]:
class SimpleEncoder(nn.Module):
    """Einfacher CNN Encoder für CIFAR-10"""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, 128)  # Feature-Dimension
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return F.normalize(self.fc(x), dim=1)  # L2-Normalisierung

class ContrastiveModel(nn.Module):
    """Self-supervised Contrastive Learning Model"""
    def __init__(self):
        super().__init__()
        self.encoder = SimpleEncoder()
        # Projection Head (wichtig für Contrastive Learning)
        self.projection = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
    
    def forward(self, x1, x2):
        # Features für beide Augmentierungen
        z1 = self.projection(self.encoder(x1))
        z2 = self.projection(self.encoder(x2))
        return F.normalize(z1, dim=1), F.normalize(z2, dim=1)

# Modell erstellen
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ContrastiveModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Modell erstellt auf: {device}")
print(f"Parameter: {sum(p.numel() for p in model.parameters())/1000:.1f}k")

## 4. Contrastive Loss

**InfoNCE Loss:** Ähnliche Paare zusammenbringen, verschiedene Paare trennen

### 💡 **Die Magie des Self-supervised Learning:**
- **Positive Paare**: Zwei Augmentierungen desselben Bildes → sollen ähnlich werden
- **Negative Paare**: Augmentierungen verschiedener Bilder → sollen verschieden bleiben  
- **InfoNCE**: Mathematische Formel, die diese Idee umsetzt
- **Ohne Labels**: Das Modell lernt nur durch diese Ähnlichkeits-/Verschiedenheits-Regel!

In [ ]:
def contrastive_loss(z1, z2, temperature=0.5):
    """InfoNCE Loss für Contrastive Learning
    
    Args:
        z1, z2: Normalisierte Features der beiden Augmentierungen
        temperature: Temperatur-Parameter für Softmax
    """
    batch_size = z1.size(0)
    
    # Similarity Matrix berechnen
    # z1 mit z2 vergleichen (positive Paare)
    pos_sim = torch.sum(z1 * z2, dim=1) / temperature  # [batch_size]
    
    # z1 mit allen anderen z2 vergleichen (negative Paare)
    neg_sim = torch.mm(z1, z2.t()) / temperature  # [batch_size, batch_size]
    
    # Diagonale entfernen (das sind die positiven Paare)
    mask = torch.eye(batch_size, device=z1.device).bool()
    neg_sim = neg_sim.masked_fill(mask, float('-inf'))
    
    # InfoNCE Loss
    logits = torch.cat([pos_sim.unsqueeze(1), neg_sim], dim=1)
    labels = torch.zeros(batch_size, device=z1.device, dtype=torch.long)
    
    loss = F.cross_entropy(logits, labels)
    
    # Accuracy berechnen (wie oft ist das positive Paar am ähnlichsten?)
    pred = torch.argmax(logits, dim=1)
    acc = (pred == labels).float().mean()
    
    return loss, acc

# Test der Loss-Funktion
with torch.no_grad():
    test_z1 = torch.randn(4, 32).to(device)
    test_z2 = torch.randn(4, 32).to(device)
    test_z1 = F.normalize(test_z1, dim=1)
    test_z2 = F.normalize(test_z2, dim=1)
    
    test_loss, test_acc = contrastive_loss(test_z1, test_z2)
    print(f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}")
    print("Loss-Funktion erfolgreich implementiert!")

## 5. Training (Self-supervised)

**Ohne Labels!** Das Modell lernt nur durch Vergleich der Augmentierungen

### 💡 **Warum funktioniert das?**
- **Pretext Task**: Erkenne, welche Augmentierungen zum gleichen Bild gehören
- **Inductive Bias**: Verschiedene Ansichten eines Objekts sollten ähnliche Eigenschaften haben
- **Emergente Features**: Das Modell entwickelt automatisch nützliche Repräsentationen
- **Skalierbar**: Funktioniert mit Millionen unlabeled Bildern!

In [ ]:
def train_self_supervised(epochs=5):
    model.train()
    losses = []
    accuracies = []
    
    print("🚀 Starte Self-supervised Training...")
    print("WICHTIG: Keine Labels verwendet - nur Augmentierungen!")
    print("-" * 50)
    
    for epoch in range(epochs):
        epoch_losses = []
        epoch_accs = []
        
        for batch_idx, (batch, _) in enumerate(train_loader):  # Labels ignorieren!
            # batch enthält tuple von (aug1, aug2)
            aug1, aug2 = batch
            aug1, aug2 = aug1.to(device), aug2.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            z1, z2 = model(aug1, aug2)
            
            # Contrastive Loss
            loss, acc = contrastive_loss(z1, z2)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(loss.item())
            epoch_accs.append(acc.item())
            
            if batch_idx % 5 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx}: Loss={loss:.3f}, Acc={acc:.3f}")
        
        avg_loss = np.mean(epoch_losses)
        avg_acc = np.mean(epoch_accs)
        losses.append(avg_loss)
        accuracies.append(avg_acc)
        
        print(f"Epoch {epoch+1} fertig: Avg Loss={avg_loss:.3f}, Avg Acc={avg_acc:.3f}")
        print("-" * 50)
    
    return losses, accuracies

# Training starten
losses, accuracies = train_self_supervised(epochs=50)

## 6. Training Visualisierung

### 💡 **Was zeigen diese Kurven?**
- **Contrastive Loss**: Fällt → das Modell wird besser im Unterscheiden ähnlich/verschieden
- **Contrastive Accuracy**: Steigt → das Modell erkennt korrekt, welche Augmentierungen zusammengehören
- **Lernfortschritt**: Ohne Labels lernt das Modell sinnvolle Repräsentationen!

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(losses, 'b-o', label='Contrastive Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Self-supervised Learning: Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(accuracies, 'r-o', label='Contrastive Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Positive Pair Recognition')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 Finaler Loss: {losses[-1]:.3f}")
print(f"📈 Finale Accuracy: {accuracies[-1]:.3f}")
print("")
print("💡 Die Accuracy zeigt, wie oft das Modell korrekt erkennt,")
print("   dass zwei Augmentierungen vom gleichen Bild stammen!")

## 7. Gelernte Repräsentationen visualisieren

**Ziel:** Zeigen, dass das Modell sinnvolle Features gelernt hat (ohne Labels!)

### 💡 **Der Moment der Wahrheit:**
- **PCA**: Reduziert 128D Features auf 2D für Visualisierung
- **Cluster-Bildung**: Wenn SSL funktioniert, sollten ähnliche Objekte clustern
- **Ohne Labels trainiert**: Das Modell hat nie gesehen, dass "Flugzeug = 0", trotzdem clustert es!
- **Emergenz**: Semantische Struktur entsteht automatisch aus dem Contrastive Learning

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Filter für 3 Klassen mit Label-Remapping
def filter_and_remap_classes(dataset, selected_classes):
    """Filtert Dataset und mappt Labels auf 0,1,2"""
    indices = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in selected_classes:
            indices.append(i)
    
    # Custom Dataset mit Label-Remapping
    class RemappedDataset(torch.utils.data.Dataset):
        def __init__(self, dataset, indices, selected_classes):
            self.dataset = dataset
            self.indices = indices
            self.class_mapping = {cls: i for i, cls in enumerate(selected_classes)}
        
        def __len__(self):
            return len(self.indices)
        
        def __getitem__(self, idx):
            data, label = self.dataset[self.indices[idx]]
            return data, self.class_mapping[label]
    
    return RemappedDataset(dataset, indices, selected_classes)

def visualize_representations():
    """Visualisiere die gelernten Repräsentationen"""
    model.eval()
    
    # Test Dataset mit Labels (nur für Visualisierung) - gefiltert auf 3 Klassen
    test_dataset_full = torchvision.datasets.CIFAR10(
        root='./data', 
        train=False, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
    )
    
    # Nur ausgewählte Klassen
    test_dataset = filter_and_remap_classes(test_dataset_full, selected_classes)
    
    # Kleine Auswahl für Visualisierung
    test_subset = torch.utils.data.Subset(test_dataset, range(min(300, len(test_dataset))))
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)
    
    features = []
    labels = []
    
    with torch.no_grad():
        for batch, batch_labels in test_loader:
            batch = batch.to(device)
            # Nur Encoder verwenden (ohne Projection Head)
            feat = model.encoder(batch)
            features.append(feat.cpu())
            labels.append(batch_labels)
    
    features = torch.cat(features, dim=0).numpy()
    labels = torch.cat(labels, dim=0).numpy()
    
    print(f"Features Shape: {features.shape}")
    
    # PCA für Dimensionsreduktion
    pca = PCA(n_components=2)
    features_2d = pca.fit_transform(features)
    
    # Visualisierung
    plt.figure(figsize=(10, 8))
    
    # Farben für 3 Klassen
    colors = ['red', 'blue', 'green']
    
    for i in range(3):
        mask = labels == i
        if mask.sum() > 0:  # Nur wenn Daten vorhanden
            plt.scatter(features_2d[mask, 0], features_2d[mask, 1], 
                       c=colors[i], label=class_names[i], alpha=0.7, s=30)
    
    plt.xlabel('PCA Komponente 1')
    plt.ylabel('PCA Komponente 2')
    plt.title('Self-supervised Repräsentationen: 3 Klassen (ohne Labels trainiert!)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("🎯 BEOBACHTE: Die 3 Klassen clustern klar zusammen!")
    print("   Das Modell hat OHNE Labels gelernt, sinnvolle Features zu extrahieren!")
    print(f"   PCA erklärt {pca.explained_variance_ratio_.sum():.2%} der Varianz")
    print(f"   Klassen: {', '.join(class_names)}")

visualize_representations()

## 8. Transfer Learning Test

**Der wahre Test:** Können die gelernten Features für eine supervised Aufgabe verwendet werden?

### 💡 **Praktischer Nutzen von SSL:**
- **Feature Reuse**: Nutze die self-supervised Features für eine neue Aufgabe
- **Frozen Encoder**: SSL-Features bleiben fest, nur der Klassifikator wird trainiert
- **Weniger Labels**: Brauche nur wenige gelabelte Daten für die neue Aufgabe
- **Industrieller Standard**: So funktionieren GPT, BERT, und moderne Vision Models!

### 📋 **Die supervised Aufgabe erklärt:**
- **Vorher (SSL)**: "Erkenne, welche Augmentierungen zusammengehören" → **ohne Labels**
- **Jetzt (Supervised)**: "Klassifiziere: Flugzeug, Auto oder Vogel?" → **mit Labels**
- **Unterschied**: Jetzt haben wir echte Klassenlabels und wollen konkrete Objekte erkennen
- **Ziel**: Testen, ob die SSL-Features auch für "normale" Klassifikation taugen

In [ ]:
class TransferClassifier(nn.Module):
    """Einfacher Klassifikator auf gefrorenen Features"""
    def __init__(self, pretrained_encoder):
        super().__init__()
        self.encoder = pretrained_encoder
        # Encoder einfrieren
        for param in self.encoder.parameters():
            param.requires_grad = False
            
        # Nur Klassifikations-Head trainieren
        self.classifier = nn.Linear(128, 3)  # 3 ausgewählte CIFAR-10 Klassen
    
    def forward(self, x):
        with torch.no_grad():
            features = self.encoder(x)
        return self.classifier(features)

# Transfer Learning Setup
transfer_model = TransferClassifier(model.encoder).to(device)
transfer_optimizer = torch.optim.Adam(transfer_model.classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Dataset mit Labels für Transfer Learning
supervised_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_supervised_full = torchvision.datasets.CIFAR10(
    root='./data', train=True, transform=supervised_transform
)
test_supervised_full = torchvision.datasets.CIFAR10(
    root='./data', train=False, transform=supervised_transform
)

# Gefilterte Datasets für Transfer Learning
train_supervised = filter_and_remap_classes(train_supervised_full, selected_classes)
test_supervised = filter_and_remap_classes(test_supervised_full, selected_classes)

# Kleine Datasets für schnelles Training
train_small = torch.utils.data.Subset(train_supervised, range(min(1000, len(train_supervised))))
test_small = torch.utils.data.Subset(test_supervised, range(min(200, len(test_supervised))))

train_loader_super = DataLoader(train_small, batch_size=64, shuffle=True)
test_loader_super = DataLoader(test_small, batch_size=64, shuffle=False)

print("🎯 Transfer Learning Setup fertig für 3 Klassen!")
print(f"   Klassen: {', '.join(class_names)}")
print(f"   Trainiere nur {sum(p.numel() for p in transfer_model.classifier.parameters())} Parameter")
print(f"   Encoder mit {sum(p.numel() for p in transfer_model.encoder.parameters())} Parametern ist eingefroren")

In [ ]:
# Transfer Learning Training
def train_transfer_learning(epochs=3):
    transfer_model.train()
    
    print("🚀 Starte Transfer Learning...")
    print("Nur der Klassifikations-Head wird trainiert!")
    print("-" * 40)
    
    for epoch in range(epochs):
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader_super):
            data, target = data.to(device), target.to(device)
            
            transfer_optimizer.zero_grad()
            output = transfer_model(data)
            loss = criterion(output, target)
            loss.backward()
            transfer_optimizer.step()
            
            # Accuracy berechnen
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
            
            if batch_idx % 5 == 0:
                acc = 100. * correct / total
                print(f"Epoch {epoch+1}, Batch {batch_idx}: Loss={loss:.3f}, Acc={acc:.1f}%")
        
        epoch_acc = 100. * correct / total
        print(f"Epoch {epoch+1} fertig: Accuracy = {epoch_acc:.1f}%")
        print("-" * 40)

# Test Funktion
def test_transfer():
    transfer_model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader_super:
            data, target = data.to(device), target.to(device)
            output = transfer_model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    
    accuracy = 100. * correct / total
    print(f"\n🎯 Test Accuracy: {accuracy:.1f}%")
    return accuracy

# Training und Test
train_transfer_learning(epochs=30)
test_accuracy = test_transfer()

## 9. Vergleich: Random vs. Self-supervised Features

### 💡 **Wissenschaftlicher Beweis:**
- **Baseline**: Zufällige Features (wie würde ein untrainiertes Modell abschneiden?)
- **Self-supervised**: Features aus unserem Contrastive Learning
- **Fairer Vergleich**: Gleiche Architektur, nur die Initialisierung unterscheidet sich
- **Erwartung**: SSL-Features sollten deutlich besser sein!

In [ ]:
# Vergleich mit zufälligen Features
random_encoder = SimpleEncoder().to(device)  # Zufällig initialisiert
random_transfer = TransferClassifier(random_encoder).to(device)
random_optimizer = torch.optim.Adam(random_transfer.classifier.parameters(), lr=0.01)

print("🎲 Teste zufällige Features als Baseline...")

# Schnelles Training nur für Vergleich
random_transfer.train()
for epoch in range(2):  # Weniger Epochen
    for batch_idx, (data, target) in enumerate(train_loader_super):
        if batch_idx > 10:  # Nur wenige Batches
            break
            
        data, target = data.to(device), target.to(device)
        random_optimizer.zero_grad()
        output = random_transfer(data)
        loss = criterion(output, target)
        loss.backward()
        random_optimizer.step()

# Test zufällige Features
random_transfer.eval()
random_correct = 0
random_total = 0

with torch.no_grad():
    for data, target in test_loader_super:
        data, target = data.to(device), target.to(device)
        output = random_transfer(data)
        pred = output.argmax(dim=1)
        random_correct += pred.eq(target).sum().item()
        random_total += target.size(0)

random_accuracy = 100. * random_correct / random_total

# Ergebnisse vergleichen
print("\n" + "="*50)
print("🏆 ERGEBNISSE VERGLEICH:")
print("="*50)
print(f"🎯 Self-supervised Features: {test_accuracy:.1f}%")
print(f"🎲 Zufällige Features:       {random_accuracy:.1f}%")
print(f"📈 Verbesserung:             +{test_accuracy - random_accuracy:.1f}%")
print("")
print("💡 FAZIT: Self-supervised Learning lernt bessere Repräsentationen!")
print("   Ohne Labels, nur durch Contrastive Learning!")

# Visualisierung des Vergleichs
plt.figure(figsize=(8, 6))
methods = ['Zufällige Features', 'Self-supervised Features']
accuracies = [random_accuracy, test_accuracy]
colors = ['lightcoral', 'lightgreen']

bars = plt.bar(methods, accuracies, color=colors, alpha=0.7, edgecolor='black')
plt.ylabel('Test Accuracy (%)')
plt.title('Transfer Learning: Zufällige vs. Self-supervised Features')
plt.ylim(0, max(accuracies) * 1.2)

# Werte auf Balken anzeigen
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Zusammenfassung & Take-Aways

### 💡 **Das Große Bild:**
Self-Supervised Learning ist **der Grund**, warum moderne KI so erfolgreich ist:
- **GPT**: Lernt Sprache ohne Labels durch "nächstes Wort vorhersagen"
- **BERT**: Lernt durch "Lücken füllen" in Texten  
- **Vision Models**: Lernen durch Contrastive Learning wie hier gezeigt
- **Skalierung**: Kann mit unbegrenzten unlabeled Daten trainiert werden

### Was haben wir gelernt?

## 🎓 **Self-Supervised Learning Zusammenfassung**

### 🔑 **Kernprinzipien:**

1. **Keine Labels erforderlich** - nur rohe Daten
2. **Pretext-Task**: Contrastive Learning - "gleiche Bilder ähnlich, verschiedene verschieden"
3. **Augmentierungen** sind entscheidend für den Erfolg
4. **Transfer Learning**: Gelernte Features für andere Aufgaben nutzen

### 📊 **Unsere Ergebnisse:**
- Self-supervised Features übertreffen zufällige Features deutlich
- Das Modell lernt sinnvolle Cluster ohne Labels
- Transfer Learning funktioniert mit wenigen gelabelten Daten

### 🚀 **Praktische Anwendungen:**

- **Medizin**: Vortraining auf unlabeled Scans, dann Fine-tuning auf spezifische Krankheiten
- **Industrie**: Qualitätskontrolle mit wenigen Labels
- **NLP**: GPT, BERT - alle nutzen Self-supervised Learning!

### ⚡ **Vorteile:**
- Nutzt große Mengen unlabeled Daten
- Bessere Features als zufällige Initialisierung
- Reduziert Bedarf an teuren Labels

### ⚠️ **Herausforderungen:**
- Augmentierungen müssen gut gewählt sein
- Rechenaufwand für Contrastive Learning
- Pretext-Task muss sinnvolle Features fördern

### 🔬 **Weiterführende Methoden:**
- **SimCLR, SwAV, BYOL** - fortgeschrittene Contrastive Learning Methoden
- **Masked Autoencoders (MAE)** - wie BERT für Bilder
- **Multimodal Learning** - Text + Bilder (wie CLIP)

---

**🎯 Zentrale Erkenntnis:** Self-supervised Learning macht unlabeled Daten nutzbar und ist ein Schlüssel für den Erfolg moderner KI-Systeme!